In [3]:
import numpy as np

In [6]:
def calculate_profit(x1,x2,y,z,n0,p1,p2,p_product):
    #参数说明
    #n:零件1的数量/零件2的数量
    #p1:零件1的次品率
    #p2:零件2的次品率
    #p_product:成品的次品率
    #一些内置的参数
    C1=8#检测零件1的成本
    C2=1#检测零件2的成本
    C_market=56#单件成品的市场售价
    C_assemble=6#装配单件成品的费用
    C_test_product=3#检测单件成品所需的费用
    C_lose=10#调换一件产品引起的损失
    C_disassemble=5#拆解一件产品所需的费用
    #更新次品率
    p1_update=(1-x1)*p1
    p2_update=(1-x2)*p2
    #成品的总次品率
    P=(1-p1_update)*(1-p2_update)*p_product+p1_update+p2_update-p1_update*p2_update
    n=np.floor(min((1-x1*p1)*n0,(1-x2*p2)*n0))
    profit=C_market*(1-P)*n-x1*C1*n0-x2*C2*n0-C_assemble*n-y*C_test_product*n-(1-y)*C_lose*n*P-z*C_disassemble*n*P
    #更新零件数
    if z==0:
        n_update=0
    elif z==1:
        n_update=np.floor(n*P)
    return profit,n_update,p1_update,p2_update

In [11]:
n0=10000
p1=0.1
p2=0.2
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.95*p1,1.05*p1,5):
    for p2 in np.linspace(0.95*p2,1.05*p2,5):
        for p_product in np.linspace(0.95*p_product,1.05*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,1,0,0]):
                same+=1
            else:
                different+=1
    

1
最终的最大利润为65828414.31268117
第一阶段的生产决策为:
[0, 0, 0, 1, 261021.27750000003, 3365.0, 0.095, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 87833.65987875, 1132.0, 0.095, 0.19]
第三阶段的生产决策为:
[0, 1, 0, 0, 33726.9754, 0, 0.095, 0.0]
2
最终的最大利润为66336940.45002011
第一阶段的生产决策为:
[0, 0, 0, 1, 259720.1137500001, 3384.0, 0.095, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 87889.28649300002, 1145.0, 0.095, 0.19]
第三阶段的生产决策为:
[0, 1, 0, 0, 33994.163275000006, 0, 0.095, 0.0]
3
最终的最大利润为66768724.49684992
第一阶段的生产决策为:
[0, 0, 0, 1, 258418.95, 3402.0, 0.095, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 87914.12679000001, 1157.0, 0.095, 0.19]
第三阶段的生产决策为:
[0, 1, 0, 0, 34221.309, 0, 0.095, 0.0]
4
最终的最大利润为67123106.37746246
第一阶段的生产决策为:
[0, 0, 0, 1, 257117.78624999992, 3420.0, 0.095, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 87934.28289749999, 1169.0, 0.095, 0.19]
第三阶段的生产决策为:
[0, 1, 0, 0, 34407.86055, 0, 0.095, 0.0]
5
最终的最大利润为67613492.67941615
第一阶段的生产决策为:
[0, 0, 0, 1, 255816.62249999994, 3439.0, 0.095, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 87975.33647774998, 1182.0, 0.095, 0.19]
第三

In [12]:
print(f"在0.95-1.05波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.95-1.05波动时,决策差异率为:{different/125}")

在0.95-1.05波动时,生产决策方案和原方案一致的次数为:0
在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:125
在0.95-1.05波动时,决策差异率为:1.0


In [13]:
n0=10000
p1=0.1
p2=0.2
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.9*p1,1.1*p1,5):
    for p2 in np.linspace(0.9*p2,1.1*p2,5):
        for p_product in np.linspace(0.9*p_product,1.1*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,1,0,0]):
                same+=1
            else:
                different+=1
    

1
最终的最大利润为61697725.53257011
第一阶段的生产决策为:
[0, 0, 0, 1, 272119.81999999995, 3209.0, 0.09000000000000001, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 87323.25023799998, 1029.0, 0.09000000000000001, 0.18000000000000002]
第三阶段的生产决策为:
[0, 1, 0, 0, 31556.8278, 0, 0.09000000000000001, 0.0]
2
最终的最大利润为62630603.698390156
第一阶段的生产决策为:
[0, 0, 0, 1, 269470.81, 3246.0, 0.09000000000000001, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 87470.22492600001, 1053.0, 0.09000000000000001, 0.18000000000000002]
第三阶段的生产决策为:
[0, 1, 0, 0, 32046.7609, 0, 0.09000000000000001, 0.0]
3
最终的最大利润为63538800.19680006
第一阶段的生产决策为:
[0, 0, 0, 1, 266821.8, 3284.0, 0.09000000000000001, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 87624.27911999999, 1078.0, 0.09000000000000001, 0.18000000000000002]
第三阶段的生产决策为:
[0, 1, 0, 0, 32523.681999999997, 0, 0.09000000000000001, 0.0]
4
最终的最大利润为64495336.387385115
第一阶段的生产决策为:
[0, 0, 0, 1, 264172.79, 3321.0, 0.09000000000000001, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 87731.78355899997, 11

In [14]:
print(f"在0.9-1.1波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.9-1.1波动时,决策差异率为:{different/125}")

在0.9-1.1波动时,生产决策方案和原方案一致的次数为:0
在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:125
在0.9-1.1波动时,决策差异率为:1.0


In [ ]:
n0=10000
p1=0.1
p2=0.2
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.8*p1,1.2*p1,5):
    for p2 in np.linspace(0.8*p2,1.2*p2,5):
        for p_product in np.linspace(0.8*p_product,1.2*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,1,0,0]):
                same+=1
            else:
                different+=1

In [15]:
print(f"在0.8-1.2波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.8-1.2波动时,决策差异率为:{different/125}")

在0.8-1.2波动时,生产决策方案和原方案一致的次数为:0
在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:125
在0.8-1.2波动时,决策差异率为:1.0
